In [85]:
import pandas as pd
import numpy as np
from pandas.core.interchange.dataframe_protocol import DataFrame
from pygments.lexer import bygroups

import volpy_func_lib as vp
import load_clean_lib
import importlib
from datetime import datetime
import matplotlib.pyplot as plt

import table_lib

from datetime import datetime

from volpy_func_lib import load_forward_price

In [86]:
# 3 cleaning bud:

# mid price for option > x  (x=~0.1)
# volumen > x
# bid ask spread / mid price < x

In [87]:
importlib.reload(vp)
importlib.reload(load_clean_lib)
importlib.reload(table_lib)

<module 'table_lib' from 'D:\\Professionelt\\Code\\volpy\\table_lib.py'>

# Lav Data

## Load, Clean and Prepare data

In [88]:
# Load data
profile = "Mads"
dataset = "i4s4"
od, FW, ZCY_curves = load_clean_lib.load_od_FW_ZCY(profile, dataset)

# first_day = pd.to_datetime("1996-01-04")
# last_day = pd.to_datetime("1997-01-01")
first_day = pd.to_datetime("1996-01-04")
# first_day = pd.to_datetime("2002-01-04")
last_day = pd.to_datetime("2003-02-28")
# last_day = pd.to_datetime("1996-05-01")

# Clean data
od = load_clean_lib.clean_od(od, first_day = first_day, last_day = last_day)

### Add FW to options

In [89]:
od = vp.add_FW_to_od(od, FW)

### remove if ITM

In [90]:
od = od.loc[
    ((od["F"] < od["K"]) & (od["cp_flag"] == "C")) | ((od["F"] > od["K"]) & (od["cp_flag"] == "P"))
]

In [91]:
# od = od[od["volume"] > 0]

### Add r to options

In [92]:
od = vp.add_r_to_od_parallel(od, ZCY_curves)

## Add low/high and create summary (Filters dataset for criteria such as min 3 strikes ... min 8 days...)

In [93]:
od, summary_dly_df = vp.od_filter_and_summary_creater(od)

### only keep the lowest ("low") and the second lowest ("high") TTMs

In [94]:
od_rdy = od[(od["low"] == True) | (od["high"] == True)]

## Calculate swap rate

In [114]:
summary_dly_df = vp.fill_swap_rates(summary_dly_df, od_rdy, n_points=200)

Processing Groups: 100%|██████████| 26686/26686 [00:11<00:00, 2293.94it/s]


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long long'

## Save as CSV

In [96]:
date_str = datetime.now().strftime("%Y-%m_%d_%H-%M")

summary_dly_df.to_csv(f"data/summary_dly_df {date_str}.csv")
od_rdy.to_csv(f"data/od_rdy {date_str}.csv")

# Load CSV

In [97]:
summary_dly_df = pd.read_csv(f"data/summary_dly_df {date_str}.csv")
od_rdy = pd.read_csv(f"data/od_rdy {date_str}.csv")

In [98]:
importlib.reload(vp)
importlib.reload(load_clean_lib)

<module 'load_clean_lib' from 'D:\\Professionelt\\Code\\volpy\\load_clean_lib.py'>

## Keep only active days, and take average for table (just overview)

In [99]:
# Select only the numeric columns in the DataFrame.
numeric_cols = summary_dly_df.select_dtypes(include=[np.number]).columns

# Filter rows where "low SW" is not NaN, group by "Ticker", and calculate the mean for numeric columns.
filtered_df = summary_dly_df[summary_dly_df["Active"] == True]
avg_by_ticker = filtered_df.groupby("Ticker")[numeric_cols].mean()
avg_by_ticker

,#days,low days,high days,low #K,high #K,#K,low SW,high SW,SW
Ticker,,,,,,,,,
AMZN,4.033307,23.758128,62.854877,11.880254,15.962728,13.921491,1.006674,0.980833,1.018289
DJX,5.850295,23.362832,53.561947,17.850295,18.008850,17.929572,0.065117,0.067193,0.065274
GE,4.591778,23.635785,61.432542,5.697742,7.723798,6.710770,0.127272,0.121146,0.124286
IBM,4.106229,23.354839,62.072859,9.722469,12.868743,11.295606,0.163329,0.151445,0.159358
MSFT,4.105673,23.356507,62.078420,8.860400,12.284205,10.572303,0.186426,0.179673,0.184420
NDX,4.696835,23.337590,53.949473,37.039423,36.615769,36.827596,0.176892,0.173006,0.175524
OEX,4.479178,23.337590,53.588007,33.555802,30.267074,31.911438,0.060220,0.061311,0.060750
SPX,5.829539,23.337590,53.603554,33.886174,29.960022,31.923098,0.058093,0.058616,0.058229


# Replications of Carr & Wu 2009 Tables

## Table 1 (Aktive dage)

In [115]:
importlib.reload(table_lib)
table_lib.CarrWu2009_table_1(summary_dly_df) # print_latex = True

,Ticker,Starting_date,Ending_date,N,NK
0,AMZN,20-Nov-1997,28-Feb-2003,1261,13.921491
1,DJX,06-Oct-1997,28-Feb-2003,1356,17.929572
2,GE,08-Jan-1996,28-Feb-2003,1727,6.710770
3,IBM,04-Jan-1996,28-Feb-2003,1798,11.295606
4,MSFT,04-Jan-1996,28-Feb-2003,1798,10.572303
5,NDX,04-Jan-1996,28-Feb-2003,1801,36.827596
6,OEX,04-Jan-1996,28-Feb-2003,1801,31.911438
7,SPX,04-Jan-1996,28-Feb-2003,1801,31.923098


## Table 2 (SW og RV)

In [116]:
table_lib.CarrWu2009_table_2(summary_dly_df) # print_latex = True

,Ticker,Mean,Std. dev.,Auto,Skew,Kurt
0,AMZN,101.828868,55.343045,0.976417,1.584801,2.940773
1,DJX,6.527394,3.430674,0.868457,1.455293,8.209319
2,GE,12.428584,6.936464,0.976598,1.278210,1.743881
3,IBM,15.935827,7.833802,0.962974,1.877459,4.400490
4,MSFT,18.442002,9.826454,0.970197,2.161156,6.921030
5,NDX,17.552373,10.816016,0.982977,0.987388,0.371159
6,OEX,6.074986,3.210560,0.964468,1.617449,2.901213
7,SPX,5.822859,3.181829,0.959574,1.813065,4.149287


## Table 3 (SW - RV og SW/RV)

In [ ]:
importlib.reload(table_lib)

## Table 4 (CAPM alpha regression)

## Table 5 (Fama-French alpha regression)

## Table 6 (Constant SW premium test)

## Table 10 (2 subsamples SW/RV ect.)